In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, Dropdown

df = pd.read_csv('concatenated_data.csv')
station_names = {
    1: 'Aotizhongxin',
    2: 'Changping',
    3: 'Dingling',
    4: 'Dongsi',
    5: 'Guanyuan',
    6: 'Gucheng',
    7: 'Huairou',
    8: 'Nongzhanguan',
    9: 'Shunyi',
    10: 'Tiantan',
    11: 'Wanliu',
    12: 'Wanshouxigong',
}
def update_heatmap(attr, station):
    
    specific_station = station
    df_filtered = df[df['station'] == specific_station]

    pivot_data = df_filtered.groupby(['year', 'month'])[attr].mean().unstack()
    normalized_data_df = pd.DataFrame(pivot_data)
    normalized_data_df = (normalized_data_df - normalized_data_df.min().min()) / (
        normalized_data_df.max().max() - normalized_data_df.min().min())
    n_years = len(pivot_data)
    n_months = len(pivot_data.columns)

    theta = np.linspace(0, 2 * np.pi, n_months + 1)
    radii = np.linspace(0.1, 1, n_years + 1)

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(8, 8))
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)

    # Create grid
    for month in range(n_months):
        ax.plot([theta[month], theta[month]], [0.1, 1], 'k', lw=0.5, alpha=0.5)
    for year in range(n_years):
        ax.plot(theta, np.full_like(theta, radii[year]), 'k', lw=0.5, alpha=0.5)

    # Create heatmap
    for year in range(n_years):
        for month in range(n_months):
            value = normalized_data_df.iloc[year, month]
            color = plt.cm.get_cmap('coolwarm')(value)
            ax.fill_between([theta[month], theta[month+1]], radii[year], radii[year+1], color=color)

    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    ax.set_xticks(theta[:-1])
    ax.set_xticklabels(month_names)
    ax.set_yticks(radii[:-1])
    ax.set_yticklabels(pivot_data.index)
    ax.yaxis.set_tick_params(pad=10)

    # Add colorbar
    cbar = fig.colorbar(plt.cm.ScalarMappable(cmap='coolwarm', norm=plt.Normalize(vmin=pivot_data.min().min(), vmax=pivot_data.max().max())),
                        ax=ax, pad=0.1, shrink=0.7, aspect=30)
    cbar.ax.set_ylabel('{} ({:.2f} - {:.2f})'.format(attr, pivot_data.min().min(), pivot_data.max().max()), rotation=270, labelpad=15)

    plt.show()

attr_dropdown = Dropdown(options=['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM'], value='PM2.5', description='Attribute:')
station_dropdown = Dropdown(
    options=[(station_names[i], i) for i in range(1, 13)],
    value=1,
    description='Station:'
)

interact(update_heatmap, attr=attr_dropdown, station=station_dropdown);


interactive(children=(Dropdown(description='Attribute:', options=('PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', '…